# General Settings and Imports

In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
#General imports
import pandas as pd
import numpy as np
import json
from pprint import pprint
from datetime import datetime, timedelta
import os

#Associated Wallets imports
from collections import deque

#Ofac imports
import csv
import requests
import xmltodict
from collections import defaultdict

#Dark Web imports
import dask.dataframe as dd
import yaml

#Flipside imports
#for flipside.xyz (SQL Databse)
#from flipside import Flipside

#Graphsense imports
import graphsense
from graphsense.api import addresses_api, bulk_api, entities_api, general_api, tags_api
from graphsense.model.neighbor_addresses import NeighborAddresses

#Coinbase imports
import hmac, hashlib, time
from requests.auth import AuthBase

#Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns
import calendar

#Mixer/Bridges imports
from collections import namedtuple
from concurrent.futures import ThreadPoolExecutor, as_completed

In [29]:
f = open('config.json')
config = json.load(f)
f.close()

CURRENCY = 'eth'
ADDRESS = '0x690B9A9E9aa1C9dB991C7721a92d351Db4FaC990'

configuration = graphsense.Configuration(
    host = config['graphsense']['host'],
    api_key = {'api_key': config['graphsense']['api_key']})

#with graphsense.ApiClient(configuration) as api_client:
    #api_instance = general_api.GeneralApi(api_client)
    #api_response = api_instance.get_statistics()
    #pprint(api_response)

# Associated Wallets Module

In [30]:
class AssociatedWallets():


    def __init__(self, currency, depth):
        self.currency = currency

    
    def simple_neighbors(self, address, direction):
        with graphsense.ApiClient(configuration) as api_client:
            # Create an instance of the API class
            api_instance = addresses_api.AddressesApi(api_client)
            #currency = CURRENCY # str | The cryptocurrency code (e.g., eth)
            #address = ADDRESS # str | The cryptocurrency address
            #direction = "out" # str | Incoming or outgoing neighbors
            #only_ids = [
                #"only_ids_example",
            #] # [str] | Restrict result to given set of comma separated addresses (optional)
            #include_labels = False # bool | Whether to include labels of first page of address tags (optional) if omitted the server will use the default value of False
            #page = "page_example" # str | Resumption token for retrieving the next page (optional)
            #pagesize = 10 # int | Number of items returned in a single page (optional)

            # example passing only required values which don't have defaults set
            try:
                # Get an address's neighbors in the address graph
                api_response = api_instance.list_address_neighbors(self.currency, address, direction)
                return api_response
            except graphsense.ApiException as e:
                print("Exception when calling AddressesApi->list_address_neighbors: %s\n" % e)
                

    #Utility function to get neighbors of address        
    def get_addr_neighbors(self, address, direction):

        if (direction == 'out'):
            degree = 'out_degree'
            address_degree = 'address_out_degree'
        else:
            degree = 'in_degree'
            address_degree = 'address_in_degree'

        with graphsense.ApiClient(configuration) as api_client:
            try:
                api_instance = bulk_api.BulkApi(api_client)
                print(f"get_addr_neighbors of {address}")
                operation = "list_address_neighbors"
                body = {'address': [address], 'direction': direction}

                df_address_neighbors = pd.read_csv(api_instance.bulk_csv(self.currency, operation, body=body,
                                                                    num_pages=1, _preload_content=False))
                
                df_address_neighbors = df_address_neighbors \
                    .loc[(df_address_neighbors['_error'] != 'not found') &
                        (df_address_neighbors['_info'] != 'no data')].reset_index(drop=True)
                
                if df_address_neighbors.empty:
                    df_address_neighbors.columns = ['address', 'entity', degree]
                    return df_address_neighbors
                
                print(df_address_neighbors.columns)  # Print column names
                
                df_address_neighbors = df_address_neighbors \
                    .rename(columns={'address_address': 'address', 
                                    'address_entity': 'entity',
                                    address_degree: degree})

                return df_address_neighbors[['address', 'entity', degree]]
                
            except graphsense.ApiException as e:
                print("Exception when calling Bulk Api: %s\n" % e)


    # The Breadth-First Search algorithm:
    def bfs_neighbors(self, seed_address, max_depth, max_degree, verbose, direction):

        if (direction == 'out'):
            degree = 'out_degree'
        else:
            degree = 'in_degree'
        
        # collect neighbors
        neighbors = []
        for i in range (0, max_depth):
            neighbors.insert(i, [])
        neighbors[0].append(seed_address)

        #keep track of depth
        levels = {seed_address: 0}
                
        # record visited addresses and entities
        visited_addresses = set([seed_address])
        
        # maintain a queue of addresses
        queue = deque([(seed_address, 0)])

        while(queue):

            # get first address from the queue
            address, level = queue.popleft()            

            # retrieve address neighbors
            df_neighbors = self.get_addr_neighbors(address, direction)

            # continue with neighbors out_degree < max_outdegree
            for index, neighbor in df_neighbors.iterrows():

                 # stop if address has already been visited
                if(neighbor['address'] in visited_addresses):
                    if verbose:
                        print(address, end=' ') 
                        print("STOP | same address")
                    continue
                                
                # stop if max depth is reached
                if level + 1 == max_depth:
                    print(level)
                    if verbose:
                        print(address, end=' ') 
                        print("STOP | max depth")
                    continue

                neighbors[level+1].append(neighbor['address'])

                # stop if address out_degree exceeds threshold
                if(neighbor[degree] > max_degree):
                    if verbose:
                        print(address, end=' ') 
                        print("STOP | max degree")
                    continue
                
                queue.append((neighbor['address'], level + 1))
                visited_addresses.add(neighbor['address'])
                levels[neighbor['address']] = level + 1
                    
            if len(queue) == 0:
                return neighbors

# Blacklist Module

In [31]:
class Blacklist():
    

    def scrape_ofac(self):

        coins = []
        myCoins = {}
        sdn = defaultdict(list)
        filename = 'sdn2.xml'
        URL = "https://www.treasury.gov/ofac/downloads/sanctions/1.0/sdn_advanced.xml"

        response = requests.get(URL)
        with open(filename, 'wb') as file:
            file.write(response.content)

        xml_data = open(filename, 'r').read()  # Read file
        d = xmltodict.parse(xml_data)

        for i in d['Sanctions']['ReferenceValueSets']['FeatureTypeValues']['FeatureType']:
            if 'Digital Currency Address' in i['#text']:
                coins.append(i['@ID'])
                myCoins[i['@ID']] =  i['#text'].replace('Digital Currency Address - ','')
            #   print(i['@ID'],'-',i['#text'])

        for i in d['Sanctions']['DistinctParties']['DistinctParty']:
            if 'Feature' in i['Profile'].keys():
                for j in i['Profile']['Feature']:
                    if '@FeatureTypeID' in j:
                        if type(j) is not str:
                            if str(j['@FeatureTypeID']) in coins:
                            #   print(j)
                            #   print(j['FeatureVersion']['VersionDetail']['#text'])
                                sdn[myCoins[j['@FeatureTypeID']]].append(j['FeatureVersion']['VersionDetail']['#text'])            
                            #    break
            
        with open('results/sdn.json', 'w') as fp:
            json.dump(sdn, fp)
        fp.close()


    def check_ofac(self, currency, address, path):
        
        file_ext = os.path.splitext(path)[1].lower()
        with open(path, "r") as file:
            if file_ext in [".json"]:
                data = json.load(file)
            else:
                raise ValueError("Unsupported file format")

        currency_set = set(data[currency])
        
        return address in currency_set
    

    def check_dark_web(self, address, parent_directory):

        #pd.set_option('display.max_colwidth', None)
    
        def scan_directory(directory):

            #Recursively scans a directory for Parquet files and checks them

            for item in os.listdir(directory):

                item_path = os.path.join(directory, item)
                
                # If item is a directory, recursively scan it
                if os.path.isdir(item_path):
                    if scan_directory(item_path):
                        return True

                # If item is a parquet file, read and check it
                elif item.endswith(".parquet"):
                    ddf = dd.read_parquet(item_path, blocksize="400mb")
                    print(ddf.head())

                    check = ddf['address'].isin([address]).compute()
                    if check.any():
                        return True
                    
            return False

        return scan_directory(parent_directory)
    
    
    def check_phishing_hack(self, address, path):

        file_ext = os.path.splitext(path)[1].lower()
        with open(path, "r") as file:
            if file_ext in [".json"]:
                data = json.load(file)
            else:
                raise ValueError("Unsupported file format")
        
        phishing_hack_addresses = [entry['address'] for entry in data]

        return address in phishing_hack_addresses

# Exchanges Module

In [32]:
class Exchange():
    

    def __init__(self, currency, depth):
        self.currency = currency
        self.depth = depth


    def get_ex_tags(self, address, type):

        # Enter a context with an instance of the API client
        with graphsense.ApiClient(configuration) as api_client:
            # Create an instance of the API class
            api_instance = addresses_api.AddressesApi(api_client)

            #currency = CURRENCY # str | The cryptocurrency code (e.g., btc)
            #address = ADDRESS # str | The cryptocurrency address
            #page = "page_example" # str | Resumption token for retrieving the next page (optional)
            #pagesize = 10 # int | Number of items returned in a single page (optional)
            #include_tags = True

            try:
                # Get attribution tags for a given address
                api_response = api_instance.list_tags_by_address(self.currency, address.lower())
                exchange_data = api_response
                
                exchange_tags = []

                for entry in exchange_data['address_tags']:
                    actor = entry.get('actor', '')
                    category = entry.get('category', '')
                    label = entry.get('label', '')
                    if ((actor or label) and (type == 'cex' and (category == 'exchange' or category == 'market'))) or ((actor or label) and type == 'dex' and category == 'defi_dex'):
                        exchange_tags.append({
                            'actor': actor,
                            'category': category,
                            'currency': entry.get('currency', ''),
                            'label': label,
                            'address': entry.get('address', '')
                        })

                return exchange_tags
                
            except graphsense.ApiException as e:
               print("Exception when calling AddressesApi->list_tags_by_address: %s\n" % e)
    
    
    def get_ex_used(self, address, type, direction, max_degree=7, verbose=True):
        
        ass_wallets = AssociatedWallets()
        neighbors = ass_wallets.bfs_neighbors(address, self.depth, max_degree, verbose, direction)

        exchanges_used = []

        for arr_level in neighbors:
            for address in arr_level:
                temp_exchange_tags = self.get_ex_tags(address, type)
                if temp_exchange_tags:
                    exchanges_used.append(temp_exchange_tags)

        return exchanges_used

# Proof of Funds Module

In [33]:
# Create custom authentication for Coinbase API
class CoinbaseWalletAuth(AuthBase):
    def __init__(self, api_key, secret_key):
        self.api_key = api_key
        self.secret_key = secret_key

    def __call__(self, request):
        timestamp = str(int(time.time()))
        message = timestamp + request.method + request.path_url + (request.body or '')
        signature = hmac.new(self.secret_key, message, hashlib.sha256).hexdigest()

        request.headers.update({
            'CB-ACCESS-SIGN': signature,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-KEY': self.api_key,
        })
        return request


class PoF():
    

    def __init__(self, currency):
        self.currency = currency

    #Candles data Coinbase

    """Each bucket is an array of the following information:

    time: bucket start time
    low: lowest price during the bucket interval
    high: highest price during the bucket interval
    open: opening price (first trade) in the bucket interval
    close: closing price (last trade) in the bucket interval
    volume: volume of trading activity during the bucket interval"""
    def coinbase_data(self, type, symbol='ETH-USD', start_date=datetime.today().strftime('%Y-%m-%d'), end_date=datetime.today().strftime('%Y-%m-%d')):

        # Before implementation, set environmental variables with the names API_KEY and API_SECRET
        #API_KEY = config['coinbase']['api_key']
        #API_SECRET = config['coinbase']['api_secret']
        #auth = CoinbaseWalletAuth(API_KEY, API_SECRET)

        # Convert the start date string to a datetime object
        #start_date_datetime = datetime.strptime(start_date, "%Y-%m-%d")

        # Calculate the end date by adding one day to the start date
        #end_date_datetime = start_date_datetime + datetime.timedelta(days=1)

        # Convert the end date to a string in the same format
        #end_date = end_date_datetime.strftime("%Y-%m-%d")

        # Parameters for the API request Exchange
        params_exchange = {
            "currency": symbol
        }

        # Parameters for the API request Spot
        params_spot = {
            "date": start_date
        }

        # Parameters for the API request Candles
        params_candles = {
            "granularity": 86400,  # Daily interval
            "start": start_date,
            "end": start_date
        }

        if type == 'buy':
            base_url = config['coinbase']['buyprice_endpoint']
            api_url = base_url.format(symbol)
            response = requests.get(api_url)

        elif type == 'sell':
            base_url = config['coinbase']['sellprice_endpoint']
            api_url = base_url.format(symbol)
            response = requests.get(api_url)

        elif type == 'exchange':
            api_url = config['coinbase']['exchangeprice_endpoint']
            response = requests.get(api_url, params=params_exchange)

        elif type == 'spot':
            base_url = config['coinbase']['spotprice_endpoint']
            api_url = base_url.format(symbol)
            response = requests.get(api_url, params=params_spot)

        elif type == 'candles':
            base_url = config['coinbase']['candles_endpoint']
            api_url = base_url.format(symbol)
            response = requests.get(api_url, params=params_candles)
        else:
            print(f"Unsupported type: {type}")
            return False

        # Process the response data
        if response.status_code == 200:
            return response.json()
        else:
            print("Request failed:", response.status_code)

    
    #Klines data Binance
    """[
        [
            1499040000000,      // Kline open time
            "0.01634790",       // Open price
            "0.80000000",       // High price
            "0.01575800",       // Low price
            "0.01577100",       // Close price
            "148976.11427815",  // Volume
            1499644799999,      // Kline Close time
            "2434.19055334",    // Quote asset volume
            308,                // Number of trades
            "1756.87402397",    // Taker buy base asset volume
            "28.46694368",      // Taker buy quote asset volume
            "0"                 // Unused field, ignore.
        ]
        ]"""
    def binance_data(self, type, symbol='', start_date=datetime.today().strftime('%Y-%m-%d'), end_date=datetime.today().strftime('%Y-%m-%d')):

        # Convert start and end dates to Unix timestamps (in milliseconds)
        start_timestamp = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp()) * 1000
        #end_timestamp = int(datetime.strptime(end_date, "%Y-%m-%d").timestamp()) * 1000

        # Parameters for the API request Klines
        params_klines = {
            "symbol": symbol,
            "interval": "1d",  # Daily interval
            "startTime": start_timestamp
            #"endTime": end_timestamp
        }

        # Parameters for the API request
        params_general = {
            "symbol": symbol,
            #"interval": "1d",  # Daily interval
            #"startTime": start_timestamp
            #"endTime": end_timestamp
        }

        if type == 'avgprice':
            api_url = config['binance']['avgprice_endpoint']
            response = requests.get(api_url, params=params_general)
        elif type == 'price':
            api_url = config['binance']['price_endpoint']
            response = requests.get(api_url, params=params_general)
        elif type == 'exchange':
            api_url = config['binance']['exchangeprice_endpoint']
            if symbol == '':
                response = requests.get(api_url)
            else:
                response = requests.get(api_url, params=params_general)
        elif type == 'klines':
            api_url = config['binance']['klines_endpoint']
            response = requests.get(api_url, params=params_klines)
        elif type == '24hrstats':
            api_url = config['binance']['24hrstats_endpoint']
            response = requests.get(api_url, params=params_general)
        else:
            print(f"Unsupported type: {type}")
            return False
        
        # Process the response data
        if response.status_code == 200:
            return response.json()
        else:
            print("Request failed:", response.status_code)
        
        
    def binance_table(self, symbol, interval='4h', limit=500, start='01-01-2023'):

        """
        interval: str tick interval - 4h/1h/1d ...
        """

        columns = ['open_time','open', 'high', 'low', 'close', 'volume','close_time', 'qav','num_trades','taker_base_vol','taker_quote_vol', 'ignore']
        start = int(datetime.timestamp(pd.to_datetime(start))*1000)
        api_url = f'https://www.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}&startTime={start}'

        data = pd.DataFrame(requests.get(api_url).json(), columns=columns, dtype=float)
        data.index = [pd.to_datetime(x, unit='ms').strftime('%Y-%m-%d %H:%M:%S') for x in data.open_time]
        usecols=['open', 'high', 'low', 'close', 'volume', 'qav','num_trades','taker_base_vol','taker_quote_vol']
        data = data[usecols]

        return data
    
    
    def check_profitloss(self, exchange, symbol, buy_date, sell_date, buy_amount, sell_amount, profit_or_loss):
        
        if exchange == 'coinbase':
            
            buy_price_data = self.coinbase_data('candles', buy_date)
            sell_price_data = self.coinbase_data('candles', sell_date)

            buy_price = float(buy_price_data[0][3])
            sell_price = float(sell_price_data[0][3])

            buy_total = buy_price * buy_amount
            sell_total = sell_price * sell_amount
            
        elif exchange == 'binance':
            
            buy_price_data = self.binance_data('klines', buy_date, symbol)
            sell_price_data = self.binance_data('klines', sell_date, symbol)

            buy_price = float(buy_price_data[0][1])
            sell_price = float(sell_price_data[0][1])

            buy_total = buy_price * buy_amount
            sell_total = sell_price * sell_amount

        else:
            print(f"Unsupported exchange: {exchange}")
            return False

        difference = abs((buy_total - sell_total) - abs(profit_or_loss))
        actual_profit_or_loss = sell_total - buy_total
        
        if difference < 1e-6: #To account for floating point inaccuracies
            print("Actual profit/loss: " + str(actual_profit_or_loss))
            print("Difference between reported profit/loss and actual profit/loss value: " + str(difference))
            return True
        else:
            print("Actual profit/loss: " + str(actual_profit_or_loss))
            print("Difference between reported profit/loss and actual profit/loss value: " + str(difference))            
            return False
        

    def get_address_txs(self, address, direction, pages=5):

        # Enter a context with an instance of the API client
        with graphsense.ApiClient(configuration) as api_client:
            # Create an instance of the API class
            api_instance = addresses_api.AddressesApi(api_client)
            #direction = "out" # str | Incoming or outgoing transactions (optional)
            #min_height = Height(1) # Height | Return transactions starting from given height (optional)
            #max_height = Height(2) # Height | Return transactions up to (including) given height (optional)
            #token_currency = "WETH" # str | Return transactions of given token currency (optional)
            #page = "page_example" # str | Resumption token for retrieving the next page (optional)
            #pagesize = 10 # int | Number of items returned in a single page (optional)

            all_txs = []
            next_page_token = ''

            for i in range(0, pages):

                try:
                    
                    if next_page_token == '':
                        # Get all transactions an address has been involved in without a next page token
                        api_response = api_instance.list_address_txs(self.currency, address, direction=direction)
                        all_txs.extend(api_response['address_txs'])

                    else:
                        # Get all transactions an address has been involved in
                        api_response = api_instance.list_address_txs(self.currency, address, direction=direction, page=next_page_token)
                        all_txs.extend(api_response['address_txs'])

                    if 'next_page' in api_response:
                        next_page_token = api_response['next_page']

                    else:
                        print(f"{i+1} pages found for transactions in direction: {direction}. Exiting after fetching all available pages.")
                        break

                except graphsense.ApiException as e:
                    print("Exception when calling AddressesApi->list_address_txs: %s\n" % e)
            
        return all_txs


    def check_pass_through_wallet(self, address, pages=5):
          
        out_txs = self.get_address_txs(address, 'out', pages)
        in_txs = self.get_address_txs(address, 'in', pages)

        params = {
            "module": "account",
            "action": "balance",
            "address": address,
            "tag": "latest",
            "apikey": config['etherscan']['api_key']
        }
        
        response = requests.get(config['etherscan']['host'], params=params)
        data = response.json()

        if data['status'] != "1" or 'result' not in data:
            print(f"Error fetching transactions for address {address}: {data['message']}")
            return []
        
        balance_eth = float(data['result']) / (10 ** 18)  # Convert wei to ether

        print("Number of incoming txs: " + str(len(in_txs['address_txs'])))
        print("Number of outgoing txs: " + str(len(out_txs['address_txs'])))
        print(balance_eth)


    def activity_analysis(self, address, pages=5):

        out_txs = self.get_address_txs(address, 'out', pages)
        in_txs = self.get_address_txs(address, 'in', pages)
        date_txcount = {}

        for tx in out_txs:
            tx_date = datetime.utcfromtimestamp(tx['timestamp']).strftime('%Y-%m-%d')
            date_txcount[tx_date] = date_txcount.get(tx_date, 0) + 1

        for tx in in_txs:
            tx_date = datetime.utcfromtimestamp(tx['timestamp']).strftime('%Y-%m-%d')
            date_txcount[tx_date] = date_txcount.get(tx_date, 0) + 1

        dates = [datetime.strptime(date, '%Y-%m-%d') for date in date_txcount.keys()]
        start_date = min(dates)
        end_date = max(dates)

        current_date = start_date
        while current_date <= end_date:
            str_date = current_date.strftime('%Y-%m-%d')
            if str_date not in date_txcount:
                date_txcount[str_date] = 0
            current_date += timedelta(days=1)

        df = pd.DataFrame(list(date_txcount.items()), columns=['Date', 'TxCount'])
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.sort_values('Date')
        df['YearMonth'] = df['Date'].dt.to_period('M')
        df['Day'] = df['Date'].dt.day
        new_df = df.pivot(index="Day", columns="YearMonth", values="TxCount")
        new_df = new_df.iloc[::-1]

        # Plot:
        #width_per_tx = 0.25
        #fig_width = len(date_txcount) * width_per_tx
        fig_width = 30
        plt.figure(figsize=(fig_width, 20))
        sns.heatmap(new_df, cmap='Purples', linewidths=2, cbar_kws={'label': 'Transaction Count'}, annot=True)
        plt.title('Transactions Heatmap')
        plt.xticks(rotation=90)
        plt.show()

# NFTs Module

In [34]:
class NFT():


    def get_nft_transfers(self, address, page=1, offset=100):

        params = {
            "module": "account",
            "action": "tokennfttx",
            #"contractaddress": "",
            "address": address,
            "page": page,
            "offset": offset,
            #"startblock": "",
            #"endblock": "",
            "sort": "asc",
            "apikey": config['etherscan']['api_key']
         }
        
        response = requests.get(config['etherscan']['host'], params=params)
        data = response.json()

        if data['status'] != "1" or 'result' not in data:
            print(f"Error fetching transactions for address {address}: {data['message']}")
            return []

        return data['result']
    
    
    def get_held_nfts(self, address):
        
        transfers = self.get_nft_transfers(address)

        incoming_nfts = {tx['tokenID']: tx for tx in transfers if tx['to'] == address.lower()}
        outgoing_nfts = {tx['tokenID']: tx for tx in transfers if tx['from'] == address.lower()}

        held_nfts = {tid: tx for tid, tx in incoming_nfts.items() if tid not in outgoing_nfts}

        return list(held_nfts.values())

# ERC20 Module

In [35]:
class ERC20():


    def get_erc20_transfers(self, address, page=3, offset=100):

        params = {
            "module": "account",
            "action": "tokentx",
            #"contractaddress": "",
            "address": address,
            "page": page,
            "offset": offset,
            #"startblock": "",
            #"endblock": "",
            "sort": "asc",
            "apikey": config['etherscan']['api_key']
         }
        
        response = requests.get(config['etherscan']['host'], params=params)
        data = response.json()

        if data['status'] != "1" or 'result' not in data:
            print(f"Error fetching transactions for address {address}: {data['message']}")
            return []

        return data['result']
    
    
    def get_erc20(self, address):
        
        transfers = self.get_erc20_transfers(address)

        erc20_balances = {}
        
        # Sum up all the incoming and outgoing transfers for each token
        for tx in transfers:

            token_key = (tx['tokenSymbol'], tx['tokenName'])
            value = float(tx['value']) / (10.0 ** float(tx['tokenDecimal']))
            
            if tx['to'] == address.lower():
                erc20_balances[token_key] = erc20_balances.get(token_key, 0) + value
            if tx['from'] == address.lower():
                erc20_balances[token_key] = erc20_balances.get(token_key, 0) - value

        # Filter out tokens with zero or negative balances
        held_erc20 = {k: v for k, v in erc20_balances.items() if v > 0}

        return dict(sorted(held_erc20.items()))

    
    def visualize(self, address):
        
        #create a chart of held erc20 tokens

        held_erc20 = self.get_erc20(address)
        pof = PoF()
        pie_data = []
        pie_labels = []
        failed_erc20 = {}

        if held_erc20:

            for token, value in held_erc20.items():

                symbol = token[0]

                if symbol != 'USDT':
                    trading_pair = symbol + 'USDT'
                    binance_call = pof.binance_data('price', trading_pair)
                    if binance_call:
                        symbol_price = float(binance_call['price'])
                        pie_data.append(symbol_price * value)
                        pie_labels.append(symbol)
                    else:
                        failed_erc20[token] = value
                elif symbol == 'USDT': 
                    symbol_price = 1.0
                    pie_data.append(symbol_price * value)
                    pie_labels.append(symbol)
    
        else:
            print(f"Error fetching transactions for address {address}")

        # Plot

        colors = [
            "#1f77b4",  # muted blue
            "#ff7f0e",  # safety orange
            "#2ca02c",  # cooked asparagus green
            "#d62728",  # brick red
            "#9467bd",  # muted purple
            "#8c564b",  # chestnut brown
            "#e377c2",  # raspberry yogurt pink
            "#7f7f7f",  # middle gray
            "#bcbd22",  # curry yellow-green
            "#17becf",  # blue-teal
            "#1a55FF",  # bright blue
            "#FF55A3",  # bright pink
            "#669900",  # olive green
            "#FFC400",  # bright yellow
            "#004DFF",  # royal blue
            "#FF5000",  # bright red-orange
            "#009966",  # teal green
            "#FF6600",  # orange
            "#8000FF",  # violet
            "#00FF80"   # mint green
        ]

        if pie_data and len(pie_data) < 6:
            plt.figure(figsize=(10, 8)) # 10 width, 8 height
            plt.legend(pie_labels, title="Tokens", loc="best")
            plt.title('ERC20 Tokens in USDT')
            plt.pie(pie_data, colors=colors)
            plt.show()
        elif pie_data and len(pie_data) >= 6:
            bar_positions = np.arange(len(pie_labels))
            plt.figure(figsize=(30, 24)) # 10 width, 8 height
            plt.xticks(bar_positions, pie_labels, rotation=45)
            plt.yscale('log')  # Set y-axis to logarithmic scale
            plt.ylabel('Value in USDT')
            plt.title('ERC20 Tokens in USDT')
            plt.bar(bar_positions, pie_data, align='center', alpha=0.7)
            plt.show()
            
        print(f"The following ERC20 Tokens are being held by the client: {held_erc20}")
        print(f"For following ERC20 Tokens there was no price data: {failed_erc20}")

# Smart Contract Module

In [36]:
class SC():


    def get_sc_data(self, contract_address, type):

        params_sc = {
            "module": "contract",
            "action": "getsourcecode",
            "address": contract_address,
            "apikey": config['etherscan']['api_key']
        }
        response_sc = requests.get(config['etherscan']['host'], params=params_sc)
        data_sc = response_sc.json()

        params_creator = {
            "module": "contract",
            "action": "getcontractcreation",
            "address": contract_address,
            "apikey": config['etherscan']['api_key']
        }
        response_creator = requests.get(config['etherscan']['host'], params=params_creator)
        data_creator = response_creator.json()

        if data_sc["status"] == "1":
            source_code = data_sc["result"][0]["SourceCode"]
            with open("results/sc_" + str(type) + "_" + data_sc["result"][0]["ContractName"] + ".txt", "w") as outfile:
                outfile.write(str(source_code))
        else:
            print(f"Error Smart Contract: {data_sc['message']}")

        if data_creator["status"] == "1":
            creator_data = data_creator["result"]
            with open("results/creator_" + str(type) + ".txt", "w") as outfile:
                outfile.write(str(creator_data))
        else:
            print(f"Error Creator: {data_creator['message']}")

# Mixer and Bridges Module

In [37]:
class MixBridge():


    def process(self, mix_bridge, address, pages, pof):

        print(f"Processing mixer/bridge: {mix_bridge['address']}")

        Tx = namedtuple('Transaction', ['from_address', 'to_address', 'name_tag'])

        out_txs = pof.get_address_txs(mix_bridge['address'], 'out', pages)
        in_txs = pof.get_address_txs(mix_bridge['address'], 'in', pages)
        #print(in_txs)

        name_tag = mix_bridge['nameTag'] if mix_bridge['nameTag'] else 'Unknown Mixer/Bridge'

        tx_from_mix_bridge = [Tx(tx['from_address'], tx['to_address'], name_tag) for tx in out_txs if tx['to_address'] == address]
        tx_to_mix_bridge = [Tx(tx['from_address'], tx['to_address'], name_tag) for tx in in_txs if tx['from_address'] == address]
        print(tx_to_mix_bridge)

        return tx_from_mix_bridge, tx_to_mix_bridge


    def check(self, address, path, pages=3):

        pof = PoF('eth')
        file_ext = os.path.splitext(path)[1].lower()
        with open(path, "r") as file:
            if file_ext in [".json"]:
                mix_bridge_data = json.load(file)
            elif file_ext in [".yaml", ".yml"]:
                mix_bridge_data = yaml.safe_load(file)
            else:
                raise ValueError("Unsupported file format")

        print(mix_bridge_data)

        tx_from_mix_bridge = []
        tx_to_mix_bridge = []

        with ThreadPoolExecutor(max_workers=8) as executor:

            futures = [executor.submit(self.process, mix_bridge, address, pages, pof) for mix_bridge in mix_bridge_data[0]]

            for future in as_completed(futures):
                out, inn = future.result()
                print("From:", out)  # Debugging print
                print("To:", inn)    # Debugging print
                tx_from_mix_bridge.extend(out)
                tx_to_mix_bridge.extend(inn)

        return tx_from_mix_bridge, tx_to_mix_bridge

# Main Section

In [38]:
# Function calls

#ass_wallets = AssociatedWallets(CURRENCY)

#direction = 'out'

#addr_neigh = ass_wallets.bfs_neighbors(ADDRESS, 4, 10, True, direction)
#addr_simple_neigh = ass_wallets.simple_neighbors(ADDRESS, direction)

#with open("results/addresses_" + direction + ".txt", "w") as outfile:
    #outfile.write(str(addr_neigh))

#blacklist = Blacklist()
#blacklist.ofac()
#print(blacklist.check_membership('ETH', ADDRESS, json_data))
#blacklist.check_dark_web('1DcU28QbeUiJVcSS8sZCf4ixeYXG26zEWN', 'data/plain_cluster_outgoing_rel.parquet')
#blacklist.check_etherscan('0x690B9A9E9aa1C9dB991C7721a92d351Db4FaC990', 'data/hack_addresses_etherscan.json')


#ex = Exchange(ADDRESS, 'eth', 3, 'in')
#exchanges_used = ex.get_ex_used('dex')
#exchanges_used = ex.get_ex_tags('0x1d42064Fc4Beb5F8aAF85F4617AE8b3b5B8Bd801', 'dex')
#pprint(exchanges_used)

#pof = PoF('eth')
#print(pof.coinbase_data('candles', '2018-01-01'))
#print(pof.binance_data('price', 'AUTOUSDT'))
#print(pof.coinbase_data('buy', 'AUTO-USD'))

#pprint(pof.binance_table('ETHUSDT', '1h'))
#pof.check_profitloss('coinbase', 'ETHUSDT', '2018-08-09', '2023-09-10', 1, 1, -300)
#print(pof.get_address_txs(ADDRESS, 'in', 1))
#pof.check_pass_through_wallet(ADDRESS, CURRENCY)
#pof.activity_analysis()

#nft = NFT()
#nfts_dict = nft.get_held_nfts(ADDRESS)
#print(nfts_dict)
#for nft in nfts_dict:
    #print(nft)

#nft_sc = nft.get_nft_sc('0x25ed58c027921e14d86380ea2646e3a1b5c55a8b')
#with open("results/nft_sc" + ".txt", "w") as outfile:
    #outfile.write(str(nft_sc))

#erc20 = ERC20(ADDRESS)
#erc20_dict = erc20.get_erc20_transfers()
#print(erc20_dict)
#print(erc20.get_erc20())
#erc20.visualize()

#sc = SC()
#sc.get_sc_data(exchanges_used[0]['address'], 'dex')

mix_bridge = MixBridge()
print(mix_bridge.check('0xA43Ce8Cc89Eff3AA5593c742fC56A30Ef2427CB0', 'data/mixers.json', 5))

[{'address': '0x94A1B5CdB22c43faab4AbEb5c74999895464Ddaf', 'nameTag': 'Tornado Cash'}, {'address': '0x8589427373D6D84E98730D7795D8f6f8731FDA16', 'nameTag': 'Tornado Cash: Donate'}, {'address': '0xdd4c48c0b24039969fc16d1cdf626eab821d3384', 'nameTag': 'Gitcoin Grants: Tornado.cash'}, {'address': '0x77777feddddffc19ff86db637967013e6c6a116c', 'nameTag': 'TORN Token (TORN)'}, {'address': '0x3efa30704d2b8bbac821307230376556cf8cc39e', 'nameTag': 'Tornado.Cash Voucher (vTORN)'}, {'address': '0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc', 'nameTag': 'Tornado.Cash: 0.1 ETH'}, {'address': '0x178169b423a011fff22b9e3f3abea13414ddd0f1', 'nameTag': 'Tornado.Cash: 0.1 WBTC'}, {'address': '0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936', 'nameTag': 'Tornado.Cash: 1 ETH'}, {'address': '0x610b717796ad172b316836ac95a2ffad065ceab4', 'nameTag': 'Tornado.Cash: 1 WBTC'}, {'address': '0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d', 'nameTag': 'Tornado.Cash: 1,00 USDC'}, {'address': '0xfd8610d20aa15b7b2e3be39b396a1bc351

KeyboardInterrupt: 

1 pages found for transactions in direction: in. Exiting after fetching all available pages.
[]
Processing mixer/bridge: 0xf60dd140cff0706bae9cd734ac3ae76ad9ebc32a
1 pages found for transactions in direction: out. Exiting after fetching all available pages.
1 pages found for transactions in direction: in. Exiting after fetching all available pages.
[]
Processing mixer/bridge: 0x07687e702b410fa43f4cb4af7fa097918ffd2730
1 pages found for transactions in direction: in. Exiting after fetching all available pages.
[]
Processing mixer/bridge: 0xd691f27f38b395864ea86cfc7253969b409c362d
1 pages found for transactions in direction: out. Exiting after fetching all available pages.
1 pages found for transactions in direction: in. Exiting after fetching all available pages.
[]
Processing mixer/bridge: 0xf67721a2d8f736e75a49fdd7fad2e31d8676542a
1 pages found for transactions in direction: in. Exiting after fetching all available pages.
[]
Processing mixer/bridge: 0xd4b88df4d29f5cedd6857912842cff3b2